In [1]:
import numpy as np
import netket as nk

[mba-10836208:54958] shmem: mmap: an error occurred while determining whether or not /var/folders/fp/xn0vr_w57pq4d84k8f4mk0480000gp/T//ompi.mba-10836208.502/jf.0/2419064832/sm_segment.mba-10836208.502.90300000.0 could be created.
/opt/homebrew/Caskroom/mambaforge/base/envs/pythonenv/lib/python3.11/site-packages/mpi4jax/_src/jax_compat.py:39: UserWarning: 
The latest supported JAX version with this release of mpi4jax is 0.4.17, but you have 0.4.30. If you encounter problems consider downgrading JAX, for example via:

    $ pip install jax[cpu]==0.4.17

Or try upgrading mpi4jax via

    $ pip install -U mpi4jax

You can set the environment variable `MPI4JAX_NO_WARN_JAX_VERSION=1` to silence this warning.
  warnings.warn(


In [12]:
def tensor_product(l):
    res = 1
    for a in l:
        res = np.kron(res, a)
    return res

In [31]:
n_qubits = 8
h = 1
J = 1

sigmax = np.array([[0,1],[1,0]])
sigmaz = np.array([[1,0],[0,-1]])

hamiltonian = 0

for k in range(n_qubits):
    hamiltonian += -h * tensor_product([np.eye(2)]*k + [sigmax] + [np.eye(2)]*(n_qubits-k-1))

for k in range(n_qubits-1):
    hamiltonian += J * tensor_product([np.eye(2)]*k + [sigmaz, sigmaz] + [np.eye(2)]*(n_qubits-k-2))

hamiltonian += J * tensor_product([sigmaz] + [np.eye(2)]*(n_qubits-2) + [sigmaz])

np.linalg.eigh(hamiltonian)[0][0]

-10.251661790966025

$$-h\sum_i \sigma_i^{(x)} +J\sum_{\langle i,j\rangle} \sigma_i^{(z)}\sigma_j^{(z)}$$

In [32]:
hilbert_space = nk.hilbert.Qubit(n_qubits)
graph = nk.graph.Chain(n_qubits, pbc=True)
hamiltonian = nk.operator.IsingJax(hilbert_space, graph, h, J)
nk.exact.lanczos_ed(hamiltonian)

array([-10.25166179])